In [ ]:
import pandas as pd
import time
import sys
def player_data(start_year,end_year,ps=False):
    if ps == False:
        season_type='leagues'
    else:
        season_type='playoffs'
    frames=[]
    for year in range(start_year,end_year+1):
        url = 'https://www.basketball-reference.com/'+season_type+'/NBA_'+str(year)+'_per_poss.html'
        df=pd.read_html(url)[0]
        df=df[df.Player!='Player']
        time.sleep(1)
        url2='https://www.basketball-reference.com/'+season_type+'/NBA_'+str(year)+'_advanced.html'
        df2=pd.read_html(url2)[0]
        df2=df2[df2.Player!='Player']

        

        year_df=df.merge(df2)
        unnamed=[col for col in year_df if 'unnamed' in col.lower()]
        year_df.drop(columns=unnamed,inplace=True)
        year_df['year']=year
        frames.append(year_df)
        print(year)

        time.sleep(2)
    return pd.concat(frames)
start_year=2000
end_year=2024
#player_stats=player_data(start_year,end_year)

#player_stats.to_csv('player_stats.csv',index=False)
player_stats_ps=player_data(start_year,end_year,ps=True)
player_stats_ps.to_csv('player_stats_ps.csv',index=False)


2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [ ]:
def league_average(league='leagues'):
    time.sleep(1)
    league_url='https://www.basketball-reference.com/'+league+'/NBA_stats_per_poss.html'
    frames=pd.read_html(league_url)
    league = frames[0]
    league.columns=league.columns.droplevel()
    league=league[league.Season!='Season']

    league2 = frames[1]
    league2.columns=league2.columns.droplevel()
    league2=league2[league2.Season!='Season']
    
    
    return league,league2
league,league2 = league_average()
league.to_csv('league_average.csv',index=False)

league2.to_csv('league_average_ps.csv',index=False)



In [ ]:
league2